In [6]:
# -*- coding:utf-8 -*-
import pymongo
import requests
import multiprocessing
from bs4 import BeautifulSoup
from time import sleep
import datetime
import random

'''
    本工程抓取房天下，广州天河区商铺出售信息
'''
def get_monggo_db():
    conn = pymongo.MongoClient('127.0.0.1',27017).locachoice
    conn.authenticate('locachoice','locachoice')
    return conn

def get_mongo_crawl_coll():
    conn = pymongo.MongoClient('127.0.0.1',27017).locachoice
    conn.authenticate('locachoice','locachoice')
    return conn['crawl_fang_shop_sale']

def clean_mongo_crawl_coll():
    db = get_monggo_db()
    re = db.drop_collection("crawl_fang_shop_sale")
    print(re)

# 请求天河商铺列表和url等基础信息
def req_tianhe_shop_list():
    base_url = 'http://shop.gz.fang.com/shou/house-a073/i3'
    coll = get_mongo_crawl_coll()
    for page_num in range(1, 53):
        page_url = '%s%d/' % (base_url, page_num)
        response = requests.get(page_url)
        html_doc = response.text        
        soup = BeautifulSoup(html_doc, 'html.parser', from_encoding = 'gb2312')
        shops = soup.find('div', class_='houseList').findAll('dl')
        for shop in shops:
            a_title = shop.find('p', class_ = 'title').find('a')
            url = 'http://shop.gz.fang.com%s' % a_title.get('href')
            title = a_title.get('title')
            p_mt15 = shop.find('p', class_ = 'mt15')            
            region = p_mt15.find('a').get('title')
            addr = p_mt15.find('span', class_ = 'iconAdress ml10 spAddr').get('title')
            desc = shop.find('p', class_ = 'mt10').text 
            area = shop.find('div', class_ = 'area alignR').text
            price = shop.find('div', class_ = 'moreInfo').find('p', class_ = 'danjia alignR mt5 gray6').text
            doc = {
                'title': title,
                'url': url,
                'region': region,
                'addr': addr,
                'desc': desc,
                'area': area,
                'price': price
            }
            print(doc)
            coll.update_one({'url': url}, {'$set': doc}, True)
    print('req_tianhe_sp_list, ended')

# 请求天河商铺信息，主要获取location
def req_tianhe_shop_info():
    coll = get_mongo_crawl_coll()
    cursor = coll.find({'lng': {'$exists': False}})
    for cur in cursor:
        try:
            print(cur.get('url'))
            html_doc = requests.get(cur.get('url')).text
            start = html_doc.find('<iframe frameborder=')
            start = html_doc.find('src=', start)
            start += 5
            end = html_doc.find('">',start)        
            map_url = html_doc[start:end]
            print(map_url)
            html_doc = requests.get(map_url).text
            start = html_doc.find('mapInfo')
            start = html_doc.find('px:', start)
            start += 4
            end = html_doc.find('",', start)
            lng = html_doc[start:end]
            print(lng)
            start = html_doc.find('py:', start)
            start += 4
            end = html_doc.find('",', start)
            lat = html_doc[start:end]
            print(lat)
            start = html_doc.find('searchInfo', end)
            start += 11
            end = html_doc.find('}', start)
            end += 1
            searchinfo = html_doc[start:end]
            print(searchinfo)
            toset_doc = {
                'map_url': map_url,
                'lng': lng,
                'lat': lat,
                'searchinfo': searchinfo,
            }
            print(toset_doc)
            coll.update_one({'_id': cur.get('_id')}, {'$set': toset_doc}, False)
        except Exception as e:
            print(e)
    
if __name__ == '__main__':
#     req_tianhe_shop_list()
    req_tianhe_shop_info()

http://shop.gz.fang.com/shou/3_199653142.html
DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd
No connection adapters were found for 'DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd'
http://shop.gz.fang.com/shou/3_199358227.html
DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd
No connection adapters were found for 'DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd'
http://shop.gz.fang.com/shou/3_199361462.html
DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd
No connection adapters were found for 'DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd'
http://shop.gz.fang.com/shou/3_193